In [1]:
!pip install category_encoders
!pip install pycaret

  Using cached category_encoders-2.2.2-py2.py3-none-any.whl (80 kB)
  Using cached pycaret-2.3.3-py3-none-any.whl (264 kB)
  Using cached cufflinks-0.17.3-py3-none-any.whl
  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached wordcloud-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (366 kB)
  Using cached plotly-5.3.1-py2.py3-none-any.whl (23.9 MB)
  Using cached mlflow-1.20.2-py3-none-any.whl (14.6 MB)
  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached pandas_profiling-3.0.0-py2.py3-none-any.whl (248 kB)
  Using cached lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached umap-learn-0.5.1.tar.gz (80 kB)
  Using cached imbalanced_learn-0.7.0-py3-none-any.whl (167 kB)
  Using cached nltk-3.6.2-py3-none-any.wh

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
from sklearn.model_selection import train_test_split
import category_encoders as ce

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pycaret.classification import *

In [3]:
import os
import psycopg2
def newCursor():
    try:
        connection = psycopg2.connect(user = os.environ["DB_USER"],
                                      host = os.environ["DB_HOST"],
                                      password = os.environ["DB_PASSWORD"],
                                      port = "5432",
                                      database = os.environ["DB_DATABASE"])

        cursor = connection.cursor()
        return cursor

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
        return error

In [4]:
c = newCursor()
q = """
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='swift'
   AND table_type='BASE TABLE'
"""
c.execute(q)
swift_tables = c.fetchall()
for table in swift_tables:
    print(table[0])

simulated_fin_parsed_hackathon
simulated_data_senders_train_hackathon


In [5]:
c = newCursor()
q = """
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='api'
   AND table_type='BASE TABLE'
"""
c.execute(q)
api_tables = c.fetchall()
for table in api_tables:
    print(table[0])

IDA_Statement_Of_Credit_and_Grants_Historical
czech_financial_data_1999_disp
ESG_by_Country_Country_Series
uk_private_consumption
wages_listing
czech_financial_data_1999_card
moody_question_codebook
czech_financial_data_1999_district
client_data_2018_2020
asic_financial_advisers
google_mobility_report_europe
low_inc_emp_small_businesses_county_daily
google_mobility_report_asia_africa
ESG_by_Country_Data
github_network_edges
London_SME
asic_self_managed_super_fund_auditor
apple_mobility_report_us
asic_business_names
new_West_Midlands_Businesses
google_mobility_report_us
asic_credit_representative
moody_raw_data
asic_credit_licensee
california_statewide_cases_covid
czech_financial_data_1999_client
czech_financial_data_1999_account
chicago_microloans
uk_10y_bond_yields
ifc_consolidated_summary_statements
banksim
california_logistics_ppe_covid
cfpb_complaints
paysim
twitter_user_data_set
asic_company
banksim_network
lendingclub_loans
ny_home_mortgage_disclosure
czech_financial_data_1999_tr

In [6]:
import pandas as pd

c = newCursor()
q = """
SELECT *
FROM swift."simulated_data_senders_train_hackathon"
"""
c.execute(q)
cols = [desc[0] for desc in c.description]
df = pd.DataFrame(c.fetchall(), columns=cols)
df.head(2)

,Transaction Ref,Originator,Sender,Receiver,Beneficiary,Date,Time,Currency,Value,Flag
0,9D7125820D5E,ARPWSTXX,ARPWSTXX,KMZBTCXX,KMZBTCXX,190726,1335,67B,10201,False
1,0D1888782D9A,ARPWSTXX,ARPWSTXX,HUAFSRXX,VLOPPGXX,190429,1259,84D,45806,False


In [7]:
##load the data from db and partition test train set and write to separate cvs
data = df.sample(frac=0.6, random_state=1).reset_index(drop=True)
data_unseen = df.drop(data.index).reset_index(drop=True)
data.to_csv('train2_2m.csv', index=False)
data_unseen.to_csv('test1.47m.csv', index=False)

In [8]:
df = pd.read_csv('train2_2m.csv')

In [9]:
df.head()

,Transaction Ref,Originator,Sender,Receiver,Beneficiary,Date,Time,Currency,Value,Flag
0,8EDA995749,BSKJPWXX,BSKJPWXX,FPNAETXX,FPNAETXX,190521,1002,385,110125,False
1,DCD43E550A,BCFTZWXX,BCFTZWXX,FAEOUAXX,EJBCBBXX,190807,1654,F8D,153004,False
2,4C3903633C9A,ARPWSTXX,ARPWSTXX,UCPWMOXX,UCPWMOXX,191225,1500,E08,11799,False
3,0BC72BAAC7BB3D5C,BIQFBTXX,BIQFBTXX,WVGTCNXX,XTEPFJXX,191021,1231,57D,118692,False
4,E49DA12ADA,BJXJBIXX,BJXJBIXX,WZKDSYXX,WZKDSYXX,190115,1335,2D3,210765,False


In [10]:
# custom features
df.drop(['Transaction Ref', 'Originator'], axis=1, inplace=True)
df['Value']=df['Value'].astype('int64')
df['Flag']= df['Flag'].astype(int)
df.Time = df.Time.apply(lambda x: str(x).zfill(4))
df['Datetime']=df['Date'].astype(str)+df['Time'].astype(str)
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%y%m%d%H%M')


df.Time = df.Time.apply(lambda x: str(x).zfill(4))
df['Datetime']=df['Date'].astype(str)+df['Time'].astype(str)
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%y%m%d%H%M')

df['mon']=df.Datetime.dt.month
# df['doy'] = df.Datetime.dt.day_of_year
df['dom']=df.Datetime.dt.day
df['dow']=df.Datetime.dt.dayofweek
df['q'] = df.Datetime.dt.quarter
df['h']=df.Datetime.dt.hour
df['m']=df.Datetime.dt.minute

######NOTE sorting below
df = df.sort_values('Datetime')
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2206077 entries, 264138 to 914844
Data columns (total 13 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Sender       object        
 1   Receiver     object        
 2   Beneficiary  object        
 3   Currency     object        
 4   Value        int64         
 5   Flag         int64         
 6   Datetime     datetime64[ns]
 7   mon          int64         
 8   dom          int64         
 9   dow          int64         
 10  q            int64         
 11  h            int64         
 12  m            int64         
dtypes: datetime64[ns](1), int64(8), object(4)
memory usage: 235.6+ MB


In [12]:
### count encoding
cat_features=['Sender', 'Receiver', 'Beneficiary','Currency']
count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(df[cat_features])
df = df.join(count_encoded.add_suffix("_count"))

##### NOTE: use this encoder for test set later. find way to persist this encoder

In [13]:
cat_features=['Sender', 'Receiver', 'Beneficiary','Currency']
target_enc = ce.CatBoostEncoder(cols=cat_features)
target_enc.fit(df[cat_features], df['Flag'])
##### NOTE: use this encoder for test set later. find way to persist this encoder
### did datetime sorting for this encoder.

# Transform the features, rename columns with _cb suffix, and join to dataframe
train_CBE = df.join(target_enc.transform(df[cat_features]).add_suffix('_cb'))
# valid_CBE = df_test.join(target_enc.transform(df_test[cat_features]).add_suffix('_cb'))

# target_enc.
# ##test
# valid_CBE = df.join(target_enc.transform(df[cat_features]).add_suffix('_cb'))

In [14]:
train_CBE.drop(['Sender', 'Receiver', 'Beneficiary','Currency','Datetime'], axis=1, inplace=True)
train_CBE

,Value,Flag,mon,dom,dow,q,h,m,Sender_count,Receiver_count,Beneficiary_count,Currency_count,Sender_cb,Receiver_cb,Beneficiary_cb,Currency_cb
264138,124806,0,1,1,1,1,4,33,131302,4523,1007,10443,0.002041,0.003316,0.000004,0.001532
1423965,215001,0,1,1,1,1,4,53,49256,399,1074,7345,0.005542,0.000009,0.000934,0.004629
688990,233236,0,1,1,1,1,4,56,74048,1873,715,22487,0.003822,0.001069,0.000005,0.004714
320842,263922,0,1,1,1,1,5,22,41383,1572,469,13990,0.006259,0.007631,0.008518,0.003217
1203284,59857,0,1,1,1,1,5,42,82353,132,369,11649,0.003036,0.000027,0.000010,0.001717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283884,21676,0,12,31,1,4,20,44,69529,100,304,8729,0.003581,0.000036,0.000012,0.003437
518216,43381,0,12,31,1,4,20,55,140986,2578,1102,7938,0.002071,0.000389,0.000003,0.001512
1952430,327340,0,12,31,1,4,20,58,189816,982,713,7905,0.001380,0.000004,0.001406,0.001898
963286,4588,0,12,31,1,4,21,3,89751,1144,1462,22601,0.003253,0.009610,0.000002,0.007168


In [15]:
setup_1 = setup(data = train_CBE, target = 'Flag', 
                session_id=123, 
                imputation_type='simple', 
                normalize=True, 
                fix_imbalance=True,
              ) 

,Description,Value
0,session_id,123
1,Target,Flag
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(2206077, 16)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [16]:
# compare_models()
rf1= create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9983,0.9772,0.5833,0.9122,0.7116,0.7108,0.7287
1,0.9981,0.9826,0.5344,0.8912,0.6682,0.6673,0.6893
2,0.9983,0.9850,0.5797,0.9014,0.7056,0.7048,0.7221
3,0.9983,0.9776,0.6116,0.8753,0.7201,0.7193,0.7309
4,0.9982,0.9817,0.5707,0.8774,0.6915,0.6907,0.7068
5,0.9983,0.9754,0.5870,0.8877,0.7067,0.7058,0.7211
6,0.9982,0.9850,0.5779,0.8861,0.6996,0.6987,0.7148
7,0.9982,0.9816,0.5670,0.8943,0.6940,0.6932,0.7113
8,0.9981,0.9808,0.5489,0.8912,0.6794,0.6785,0.6986
9,0.9982,0.9839,0.5707,0.8898,0.6954,0.6945,0.7118


In [17]:
predict_model(rf1);

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.9983,0.9805,0.6041,0.8928,0.7206,0.7197,0.7336


In [18]:
save_model(rf1,'rf7_cb2')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Flag',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strateg...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='auto',
                                         max_l

In [19]:
# load saved model
rf = load_model('rf7_cb2')

Transformation Pipeline and Model Successfully Loaded


In [20]:
test = pd.read_csv('test1.47m.csv')
test=test.sample(frac=0.7, random_state=1)
test.shape

(1029503, 10)

In [21]:
# applying same tranformation to test. this needs to go to pipeline
test.drop(['Transaction Ref', 'Originator'], axis=1, inplace=True)
test['Value']=test['Value'].astype('int64')

test.Time = test.Time.apply(lambda x: str(x).zfill(4))
test['Datetime']=test['Date'].astype(str)+test['Time'].astype(str)
test['Datetime'] = pd.to_datetime(test['Datetime'], format='%y%m%d%H%M')
test.Time = test.Time.apply(lambda x: str(x).zfill(4))
test['Datetime']=test['Date'].astype(str)+test['Time'].astype(str)
test['Datetime'] = pd.to_datetime(test['Datetime'], format='%y%m%d%H%M')

test['mon']=test.Datetime.dt.month
# test['doy'] = test.Datetime.dt.day_of_year
test['dom']=test.Datetime.dt.day
test['dow']=test.Datetime.dt.dayofweek
test['q'] = test.Datetime.dt.quarter
test['h']=test.Datetime.dt.hour
test['m']=test.Datetime.dt.minute

# y_test['Datetime']=test['Datetime']
#########################NOTE this sorting
test = test.sort_values('Datetime')
# y_test=y_test.sort_values('Datetime')

test.drop(['Date', 'Time'], axis=1, inplace=True)

##################using encoders trained on train set##############################
cat_features=['Sender', 'Receiver', 'Beneficiary','Currency']
count_enc1 = ce.CountEncoder() #### not using this
# count_encoded = count_enc1.fit_transform(test[cat_features])
count_encoded = count_enc.transform(test[cat_features])
test = test.join(count_encoded.add_suffix("_count"))

valid_CBE = test.join(target_enc.transform(test[cat_features]).add_suffix('_cb'))


valid_CBE.drop(['Sender', 'Receiver', 'Beneficiary','Currency','Datetime'], axis=1, inplace=True)
# y_test

In [22]:
valid_CBE.columns

Index(['Value', 'Flag', 'mon', 'dom', 'dow', 'q', 'h', 'm', 'Sender_count',
       'Receiver_count', 'Beneficiary_count', 'Currency_count', 'Sender_cb',
       'Receiver_cb', 'Beneficiary_cb', 'Currency_cb'],
      dtype='object')

In [23]:
pred = predict_model(rf1, data=valid_CBE)

In [24]:
pred = predict_model(rf1, data=valid_CBE)
from sklearn.metrics import f1_score

f1 = f1_score(valid_CBE.Flag, pred.Label)
f1

0.7462965850615937

In [25]:
import sklearn
sklearn.metrics.matthews_corrcoef(valid_CBE.Flag, pred.Label, sample_weight=None )

0.7624449557152361

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(valid_CBE.Flag, pred.Label)

array([[1025483,     144],
       [   1483,    2393]])

In [27]:
valid_CBE.Flag.sum(), pred.Label.sum()

(3876, 2537)